In [1]:
import fgspectra.frequency as fgf
import fgspectra.power as fgp
import fgspectra.cross as fgc
import numpy as np
import matplotlib.pyplot as plt

nu = np.array([100.,200.,400.])
ells = np.linspace(2,10,9)

In [2]:
sed = fgf.ModifiedBlackBody(nu=nu, nu_0=500.)
sed_diff = sed.diff(beta=3., temp=12)
sed_diff

{'beta': array([[-0.00050076, -0.00365646, -0.03961693]]),
 'temp': array([[-2.85085325e-05, -2.86016409e-04, -4.56012232e-03]])}

In [4]:
sed = fgf.FreeFree(nu=nu)
sed_diff = sed.diff(Te=7000, EM=600)
sed_diff

{'EM': array([[14.55139965, 14.16924818, 13.787097  ]]),
 'Te': array([[0.07088505, 0.07088501, 0.07088495]])}

In [6]:
import sps4lat.model as mod
ell = np.arange(10)


In [8]:
cl = mod.CMB(ell=ell)

OSError: ../data/cmb.dat not found.

In [ ]:
a_diff = np.eye(3)
a = np.ones(3)
cls = np.arange(15)
diff = np.einsum('p...i,...j,...l->p...ijl',a_diff,a,cls) + np.einsum('...i,p...j,l->p...ijl',a,a_diff,cls)
diff.shape

In [ ]:
a = np.zeros((3, 3, 3))
np.fill_diagonal(a,np.array([1,2,3]))
b = np.broadcast_to(a,(25,3,3,3)).T
b.shape
b[0,...,10]